## Ejercicio: Visualización y análisis de predicciones en España

En el ejercicio 2 hacíamos un vistazo inicial y respondíamos a ciertas preguntas sobre el dataset. En este ejercicio se utilizarán gráficos para responder a las siguientes preguntas:

1. Como se distribuyen las ventas realizadas en:
    - Cada país
    - Cada mes y año
    - Cada marca
2. Cual es la tendencia y estacionalidad de:
    - Todas las ventas del país con menos ventas
    - La marca con más ventas
3. Cuales son las predicciones hechas en España y como de
buenas son.

### 0. Análisis y limpieza inicial

In [353]:
import pandas as pd

df = pd.read_csv("datasets/datos_ejercicio_ventas.csv")
df.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


El dataset cuenta con los siguientes campos:
- COUNTRY: País en el que se realiza la operación (Ejemplo: Portugal)

- SUBBRAND: Producto del que se tiene el dato (Ejemplo: Lipton (L3))

- YEAR and MONTH: Cada par nos indica un instante en el tiempo (Ejemplo: 2023-12) 

- SCENARIO: Tipo de dato (Predicción o actual)

- FORECAST: En caso de existir nos dice el mes en el que se hace la predicción (AI_P02F se refiere a predicciones hechas en enero)

- FORECAST_YEAR: Año en el que se realiza la predicción (Para este dataset solo se han hecho predicciones en 2023)

- AMOUNT: Cantidad (Estimada o real en función de si es predicción o actual)


In [354]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18666 entries, 0 to 18665
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   COUNTRY        18666 non-null  object 
 1   SUBBRAND       18666 non-null  object 
 2   YEAR           18666 non-null  int64  
 3   MONTH          18666 non-null  int64  
 4   SCENARIO       18666 non-null  object 
 5   FORECAST       17766 non-null  object 
 6   FORECAST_YEAR  17766 non-null  float64
 7   AMOUNT         18666 non-null  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 1.1+ MB


Primero se comprueba la existencia de duplicados

In [355]:
duplicates = df.duplicated()
duplicates.sum()

np.int64(663)

Para este tipo de datos podemos eliminar los duplicados (no nos interesa la frecuencia con la que aparecen datos).

In [356]:
df = df.loc[~duplicates, :]
df.duplicated().sum()

np.int64(0)

Existen valores NA que hacen referencia a que no se tienen datos de predicción cuando se trata de un actual.

In [357]:
na_df = pd.DataFrame({'NA_Counts': df.isna().sum()})
na_df

,NA_Counts
COUNTRY,0
SUBBRAND,0
YEAR,0
MONTH,0
SCENARIO,0
FORECAST,900
FORECAST_YEAR,900
AMOUNT,0


In [358]:
scenario_count = df.loc[:, "SCENARIO"].value_counts().to_frame()
scenario_count.index.name = None
scenario_count

,count
AI_forecast,17103
actual,900


Podemos visualizar el porcentaje de cada tipo

In [359]:
import plotly.express as px
fig = px.pie(
    scenario_count,
    values="count",
    names=scenario_count.index,
    title='Predicciones vs Actuals'
)
fig.show()

Dividimos en 2 el dataset, uno que contiene los actuals y otro que contiene las predicciones:

In [360]:
actuals_mask = df.loc[:, "SCENARIO"] == "actual"
df_actuals = df.loc[actuals_mask, :].reset_index(drop=True)
df_actuals.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Pepsi Max (L3),2023,10,actual,NaN,NaN,1.885949e+05
1,Portugal,7up (L3),2023,3,actual,NaN,NaN,2.934971e+05
2,Portugal,7up (L3),2023,10,actual,NaN,NaN,3.484466e+05
3,Great Britain,7up Free (L3),2023,10,actual,NaN,NaN,1.172553e+06
4,Norway,Pepsi Regular (L3),2023,10,actual,NaN,NaN,3.784859e+04


De `df_actuals` podemos eliminar las columnas de predicción (`AMOUNT` es el valor real) y la columna `SCENARIO` al tratarse este subconjunto solo de actuals

In [361]:
df_actuals = df_actuals.dropna(axis=1, how="all")
df_actuals = df_actuals.drop(columns="SCENARIO")
df_actuals.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,AMOUNT
0,Portugal,Pepsi Max (L3),2023,10,1.885949e+05
1,Portugal,7up (L3),2023,3,2.934971e+05
2,Portugal,7up (L3),2023,10,3.484466e+05
3,Great Britain,7up Free (L3),2023,10,1.172553e+06
4,Norway,Pepsi Regular (L3),2023,10,3.784859e+04


Para las predicciones la máscara es inversa:

In [362]:
df_forecasts = df.loc[~actuals_mask, :]
df_forecasts.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


De nuevo la columna SCENARIO no nos aporta información (con el nombre de la variable nos basta)

In [363]:
df_forecasts = df_forecasts.drop(columns="SCENARIO")
df_forecasts.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_P03F,2023.0,396176.120491


Otro formato útil es convertir las fechas a tipo datetime, sustituyendo `MONTH` y `YEAR` por `DATE`.

In [364]:
df_actuals["DATE"] = pd.to_datetime(df_actuals[['YEAR', 'MONTH']].assign(DAY=1))
df_actuals = df_actuals.drop(columns=["YEAR", "MONTH"])
df_actuals.head()

,COUNTRY,SUBBRAND,AMOUNT,DATE
0,Portugal,Pepsi Max (L3),1.885949e+05,2023-10-01
1,Portugal,7up (L3),2.934971e+05,2023-03-01
2,Portugal,7up (L3),3.484466e+05,2023-10-01
3,Great Britain,7up Free (L3),1.172553e+06,2023-10-01
4,Norway,Pepsi Regular (L3),3.784859e+04,2023-10-01


In [365]:
df_forecasts["DATE"] = pd.to_datetime(df_forecasts[['YEAR', 'MONTH']].assign(DAY=1))
df_forecasts = df_forecasts.drop(columns=["YEAR", "MONTH"]).reset_index(drop=True)
df_forecasts.head()

,COUNTRY,SUBBRAND,FORECAST,FORECAST_YEAR,AMOUNT,DATE
0,Portugal,Lipton (L3),AI_P02F,2023.0,754356.237194,2023-12-01
1,Great Britain,Lipton (L3),AI_P10F,2023.0,560030.558029,2023-12-01
2,Spain,Pepsi Max (L3),AI_P09F,2023.0,88501.980847,2023-12-01
3,Great Britain,7up (L3),AI_P10F,2023.0,363224.511516,2024-12-01
4,Hungary,Lipton (L3),AI_P03F,2023.0,396176.120491,2023-09-01


Para el caso de forecasts también se convierte a datetime cuando se ha hecho la predicción combinando las columnas 

In [366]:
forecast_ai_date = df_forecasts["FORECAST"].unique()
print(forecast_ai_date)
print(f"Número de valores posibles = {len(forecast_ai_date)}")

['AI_P02F' 'AI_P10F' 'AI_P09F' 'AI_P03F' 'AI_PF' 'AI_P11F' 'AI_P06F'
 'AI_P05F' 'AI_P07F' 'AI_P12F' 'AI_P08F' 'AI_P04F']
Número de valores posibles = 12


Se tiene en cuenta lo visto en clase, esto es que `AI_PNF` se refiere a predicciones hechas en el mes anterior al mes `N`. Y `AI_PF` se refiere predicciones hechas al final de el año (Diciembre). Le podemos pedir a la IA que nos genere una función que cree el mapeado a una nueva columna `FORECAST_MONTH` utilizando expresiones regulares.

In [367]:
import re
import numpy as np

# Función para extraer el número y restarle 1, o devolver 12 si es "AI_PF"
def extraer_valor(cadena):
    # Verificar si la cadena es "AI_PF"
    if cadena == 'AI_PF':
        return 12
    # Usar expresión regular para extraer el número entre "P" y "F"
    match = re.search(r'P(\d+)F', cadena)
    if match:
        return int(match.group(1)) - 1
    return None  # En caso de que no se encuentre un patrón válido

np.sort(df_forecasts['FORECAST'].apply(extraer_valor).unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

Y aplicamos la función para sustituir `FORECAST` y `FORECAST_YEAR` por `FORECAST_DATE` (pasando por `FORECAST_MONTH`)

In [368]:
# Columna que indica el mes en el que se hace la predicción
df_forecasts['FORECAST_MONTH'] = df_forecasts['FORECAST'].apply(extraer_valor)
df_forecasts["FORECAST_DATE"] = pd.to_datetime(
    df_forecasts[["FORECAST_MONTH", "FORECAST_YEAR"]]
    .assign(DAY=1)
    .rename(columns={"FORECAST_MONTH": "MONTH", "FORECAST_YEAR": "YEAR"}) # Necesario (pandas es inconsistente aquí)
)
df_forecasts = df_forecasts.drop(columns=["FORECAST_MONTH", "FORECAST_YEAR", "FORECAST"])
print(df_forecasts.size)
df_forecasts.head()

85515


,COUNTRY,SUBBRAND,AMOUNT,DATE,FORECAST_DATE
0,Portugal,Lipton (L3),754356.237194,2023-12-01,2023-01-01
1,Great Britain,Lipton (L3),560030.558029,2023-12-01,2023-09-01
2,Spain,Pepsi Max (L3),88501.980847,2023-12-01,2023-08-01
3,Great Britain,7up (L3),363224.511516,2024-12-01,2023-09-01
4,Hungary,Lipton (L3),396176.120491,2023-09-01,2023-02-01


Finalmente podemos hacernos una idea de la magnitud del horizonte de predicción (forecast) y la línea temporal tanto de ventas reales (actuals) que se han medido como instantes en los que se realizaron predicciones.

In [369]:
actuals_dates_registered = df_actuals.groupby(["COUNTRY", "SUBBRAND"])["DATE"].agg(
    initial_date='min',  # Fecha inicial (mínima)
    final_date='max',    # Fecha final (máxima)
    num_dates='count'    # Número de fechas (tamaño del grupo)
).sort_values(by="num_dates", ascending=False)
actuals_dates_registered

initial_date final_date  num_dates
COUNTRY       SUBBRAND                                             
Czech         7up (L3)             2023-01-01 2024-08-01         20
              Lipton (L3)          2023-01-01 2024-08-01         20
              Pepsi Max (L3)       2023-01-01 2024-08-01         20
              Mountain Dew (L3)    2023-01-01 2024-08-01         20
              Pepsi Regular (L3)   2023-01-01 2024-08-01         20
Denmark       7up Free (L3)        2023-01-01 2024-08-01         20
Great Britain 7up (L3)             2023-01-01 2024-08-01         20
Denmark       Mountain Dew (L3)    2023-01-01 2024-08-01         20
              Pepsi Max (L3)       2023-01-01 2024-08-01         20
              Pepsi Regular (L3)   2023-01-01 2024-08-01         20
Great Britain Lipton (L3)          2023-01-01 2024-08-01         20
              7up Free (L3)        2023-01-01 2024-08-01         20
              Mountain Dew (L3)    2023-01-01 2024-08-01         20
              Pepsi Max (L3)       2023-01-01 2024-08-01         20
Spain         Pepsi Max (L3)       2023-01-01 2024-08-01         20
Great Britain Pepsi Regular (L3)   2023-01-01 2024-08-01         20
Hungary       7up Free (L3)        2023-01-01 2024-08-01         20
              Lipton (L3)          2023-01-01 2024-08-01         20
              Pepsi Max (L3)       2023-01-01 2024-08-01         20
              Mountain Dew (L3)    2023-01-01 2024-08-01         20
              Pepsi Regular (L3)   2023-01-01 2024-08-01         20
Italy         7up (L3)             2023-01-01 2024-08-01         20
Netherlands   7up Free (L3)        2023-01-01 2024-08-01         20
Italy         Pepsi Max (L3)       2023-01-01 2024-08-01         20
              Pepsi Regular (L3)   2023-01-01 2024-08-01         20
Netherlands   7up (L3)             2023-01-01 2024-08-01         20
              Pepsi Max (L3)       2023-01-01 2024-08-01         20
              Lipton (L3)          2023-01-01 2024-08-01         20
              Pepsi Regular (L3)   2023-01-01 2024-08-01         20
Norway        7up (L3)             2023-01-01 2024-08-01         20
Spain         Pepsi Regular (L3)   2023-01-01 2024-08-01         20
Norway        7up Free (L3)        2023-01-01 2024-08-01         20
              Mountain Dew (L3)    2023-01-01 2024-08-01         20
              Pepsi Max (L3)       2023-01-01 2024-08-01         20
Portugal      7up (L3)             2023-01-01 2024-08-01         20
Norway        Pepsi Regular (L3)   2023-01-01 2024-08-01         20
Portugal      7up Free (L3)        2023-01-01 2024-08-01         20
              Pepsi Max (L3)       2023-01-01 2024-08-01         20
Spain         Lipton (L3)          2023-01-01 2024-08-01         20
Portugal      Pepsi Regular (L3)   2023-01-01 2024-08-01         20
Spain         7up (L3)             2023-01-01 2024-08-01         20
              7up Free (L3)        2023-01-01 2024-08-01         20
Portugal      Lipton (L3)          2023-01-01 2024-08-01         19
Norway        Lipton (L3)          2023-02-01 2024-08-01         18
Hungary       7up (L3)             2023-01-01 2024-07-01         16
Czech         7up Free (L3)        2024-02-01 2024-08-01          7

No obstante hay que tener cuidado, podría haber actuals con distintos `AMOUNTS`. Para comprobarlo comparamos el número de fechas posibles con las mediciones que realmente se han hecho.

In [370]:
actuals_dates_registered['month_diff'] = (actuals_dates_registered["final_date"].dt.to_period("M") - actuals_dates_registered["initial_date"].dt.to_period("M")).apply(lambda x: x.n + 1)
actuals_dates_registered.head()

initial_date final_date  num_dates  month_diff
COUNTRY SUBBRAND                                                         
Czech   7up (L3)             2023-01-01 2024-08-01         20          20
        Lipton (L3)          2023-01-01 2024-08-01         20          20
        Pepsi Max (L3)       2023-01-01 2024-08-01         20          20
        Mountain Dew (L3)    2023-01-01 2024-08-01         20          20
        Pepsi Regular (L3)   2023-01-01 2024-08-01         20          20

Comprobamos que para cada mes existe al menos y unicamente una medición.

In [371]:
inconsistent_actuals_mask = actuals_dates_registered["num_dates"] != actuals_dates_registered["month_diff"]
actuals_dates_registered.loc[inconsistent_actuals_mask, :]

,,initial_date,final_date,num_dates,month_diff
COUNTRY,SUBBRAND,,,,
Portugal,Lipton (L3),2023-01-01,2024-08-01,19,20
Norway,Lipton (L3),2023-02-01,2024-08-01,18,19
Hungary,7up (L3),2023-01-01,2024-07-01,16,19


No es el caso de Portugal, Noruega y Hungría, para visualizar actuals no es demasiado problema simplemente faltarán mediciones en algunos meses de ventas reales.

También comprobamos la magnitud de medidas en cuanto a instantes de tiempo que se predicen.

In [372]:
forecasts_dates_registered = df_forecasts.groupby(["COUNTRY", "SUBBRAND", "FORECAST_DATE"])["DATE"].agg(
    initial_date='min',  # Fecha inicial (mínima)
    final_date='max',    # Fecha final (máxima)
    num_dates='count'    # Número de fechas (tamaño del grupo)
).sort_values(by="num_dates", ascending=False)
forecasts_dates_registered.head()

initial_date final_date  num_dates
COUNTRY SUBBRAND       FORECAST_DATE                                   
Denmark Pepsi Max (L3) 2023-08-01      2023-09-01 2025-02-01        180
                       2023-10-01      2023-11-01 2025-04-01        180
                       2023-11-01      2023-12-01 2025-05-01        180
                       2023-09-01      2023-10-01 2025-03-01        180
                       2023-07-01      2023-08-01 2025-01-01        180

En el output anterior vemos que hay 180 fechas lo cual es exagerado, y podríamos no darnos cuenta o no verlo por el tamaño del output. Realizamos la misma comprobación de `num_dates` frente a la diferencia existente entre meses (`month_diff`)

In [373]:
forecasts_dates_registered['month_diff'] = (forecasts_dates_registered["final_date"].dt.to_period("M") - forecasts_dates_registered["initial_date"].dt.to_period("M")).apply(lambda x: x.n + 1)
forecasts_dates_registered.head()

initial_date final_date  num_dates  \
COUNTRY SUBBRAND       FORECAST_DATE                                      
Denmark Pepsi Max (L3) 2023-08-01      2023-09-01 2025-02-01        180   
                       2023-10-01      2023-11-01 2025-04-01        180   
                       2023-11-01      2023-12-01 2025-05-01        180   
                       2023-09-01      2023-10-01 2025-03-01        180   
                       2023-07-01      2023-08-01 2025-01-01        180   

                                      month_diff  
COUNTRY SUBBRAND       FORECAST_DATE              
Denmark Pepsi Max (L3) 2023-08-01             18  
                       2023-10-01             18  
                       2023-11-01             18  
                       2023-09-01             18  
                       2023-07-01             18

In [374]:
inconsistent_forecasts_mask = forecasts_dates_registered["num_dates"] != forecasts_dates_registered["month_diff"]
forecasts_dates_registered.loc[inconsistent_forecasts_mask, :]

initial_date final_date  num_dates  \
COUNTRY SUBBRAND       FORECAST_DATE                                      
Denmark Pepsi Max (L3) 2023-08-01      2023-09-01 2025-02-01        180   
                       2023-10-01      2023-11-01 2025-04-01        180   
                       2023-11-01      2023-12-01 2025-05-01        180   
                       2023-09-01      2023-10-01 2025-03-01        180   
                       2023-07-01      2023-08-01 2025-01-01        180   
...                                           ...        ...        ...   
Hungary Lipton (L3)    2023-11-01      2023-12-01 2025-05-01         36   
                       2023-12-01      2023-01-01 2024-06-01         36   
Norway  Lipton (L3)    2023-11-01      2023-12-01 2025-05-01         29   
Hungary 7up (L3)       2023-10-01      2023-11-01 2025-04-01         27   
                       2023-11-01      2023-12-01 2025-05-01         25   

                                      month_diff  
COUNTRY SUBBRAND       FORECAST_DATE              
Denmark Pepsi Max (L3) 2023-08-01             18  
                       2023-10-01             18  
                       2023-11-01             18  
                       2023-09-01             18  
                       2023-07-01             18  
...                                          ...  
Hungary Lipton (L3)    2023-11-01             18  
                       2023-12-01             18  
Norway  Lipton (L3)    2023-11-01             18  
Hungary 7up (L3)       2023-10-01             18  
                       2023-11-01             18  

[219 rows x 4 columns]

En este caso hay differencias que son problemáticas, al haber más predicciones que número de meses para hacerlas. Esto significa que existen varias predicciones de `AMOUNT` realizadas en un mismo instante (`AI_PNF`) para un cierto mes de un cierto año. Si son menos no hay problema:

In [375]:
ok_forecasts_mask = forecasts_dates_registered["num_dates"] <= forecasts_dates_registered["month_diff"]
forecasts_dates_registered.loc[ok_forecasts_mask, :].head()

initial_date final_date  num_dates  \
COUNTRY  SUBBRAND          FORECAST_DATE                                      
Portugal 7up Free (L3)     2023-04-01      2023-05-01 2024-10-01         18   
Norway   Mountain Dew (L3) 2023-06-01      2023-07-01 2024-12-01         18   
                           2023-05-01      2023-06-01 2024-11-01         18   
                           2023-04-01      2023-05-01 2024-10-01         18   
                           2023-03-01      2023-04-01 2024-09-01         18   

                                          month_diff  
COUNTRY  SUBBRAND          FORECAST_DATE              
Portugal 7up Free (L3)     2023-04-01             18  
Norway   Mountain Dew (L3) 2023-06-01             18  
                           2023-05-01             18  
                           2023-04-01             18  
                           2023-03-01             18

Mientras que no son válidos el negado de la máscara `ok_forecasts_mask`

In [376]:
not_valid_forecasts = forecasts_dates_registered.loc[~ok_forecasts_mask, :]
not_valid_forecasts.head()

initial_date final_date  num_dates  \
COUNTRY SUBBRAND       FORECAST_DATE                                      
Denmark Pepsi Max (L3) 2023-08-01      2023-09-01 2025-02-01        180   
                       2023-10-01      2023-11-01 2025-04-01        180   
                       2023-11-01      2023-12-01 2025-05-01        180   
                       2023-09-01      2023-10-01 2025-03-01        180   
                       2023-07-01      2023-08-01 2025-01-01        180   

                                      month_diff  
COUNTRY SUBBRAND       FORECAST_DATE              
Denmark Pepsi Max (L3) 2023-08-01             18  
                       2023-10-01             18  
                       2023-11-01             18  
                       2023-09-01             18  
                       2023-07-01             18

Podríamos tener menos predicciones pero no más. Una solución posible sería sustituir por la media de `AMOUNT`, como no conocemos los datos tal vez no sea adecuado, pero lo realizamos por ser sencillo.

In [377]:
df_forecasts = df_forecasts.groupby(["COUNTRY", "SUBBRAND", "DATE", "FORECAST_DATE"])["AMOUNT"].mean().reset_index()
df_forecasts.head()

,COUNTRY,SUBBRAND,DATE,FORECAST_DATE,AMOUNT
0,Czech,7up (L3),2023-01-01,2023-12-01,50431.232675
1,Czech,7up (L3),2023-02-01,2023-01-01,49332.089436
2,Czech,7up (L3),2023-02-01,2023-12-01,47974.059972
3,Czech,7up (L3),2023-03-01,2023-01-01,53250.490617
4,Czech,7up (L3),2023-03-01,2023-02-01,58516.218078


De esta forma aquellos elementos agrupados por `COUNTRY`, `SUBBRAND`, `DATE`, `FORECAST_DATE` que tienen distintos `AMOUNT` se ven sustituidos por una sola fila cuyo `AMOUNT` es la media.

Comprobamos si lo hemos hecho bien:

In [378]:
forecasts_dates_registered = df_forecasts.groupby(["COUNTRY", "SUBBRAND", "FORECAST_DATE"])["DATE"].agg(
    initial_date='min',  # Fecha inicial (mínima)
    final_date='max',    # Fecha final (máxima)
    num_dates='count'    # Número de fechas (tamaño del grupo)
).sort_values(by="num_dates", ascending=False)

forecasts_dates_registered['month_diff'] = (forecasts_dates_registered["final_date"].dt.to_period("M") - forecasts_dates_registered["initial_date"].dt.to_period("M")).apply(lambda x: x.n + 1)

forecasts_dates_registered.head()

initial_date final_date  num_dates  \
COUNTRY SUBBRAND           FORECAST_DATE                                      
Spain   Pepsi Regular (L3) 2023-12-01      2023-01-01 2024-06-01         18   
Czech   7up (L3)           2023-01-01      2023-02-01 2024-07-01         18   
                           2023-02-01      2023-03-01 2024-08-01         18   
                           2023-03-01      2023-04-01 2024-09-01         18   
Spain   Pepsi Max (L3)     2023-08-01      2023-09-01 2025-02-01         18   

                                          month_diff  
COUNTRY SUBBRAND           FORECAST_DATE              
Spain   Pepsi Regular (L3) 2023-12-01             18  
Czech   7up (L3)           2023-01-01             18  
                           2023-02-01             18  
                           2023-03-01             18  
Spain   Pepsi Max (L3)     2023-08-01             18

In [379]:
ok_forecasts_mask = forecasts_dates_registered["num_dates"] <= forecasts_dates_registered["month_diff"]
not_valid_forecasts = forecasts_dates_registered.loc[~ok_forecasts_mask, :]
print(f"Número de forecasts no válido: {not_valid_forecasts.size}")
not_valid_forecasts

Número de forecasts no válido: 0


,,,initial_date,final_date,num_dates,month_diff
COUNTRY,SUBBRAND,FORECAST_DATE,,,,


Hemos realizado la limpieza correctamente

### 1.1. Distribución de ventas por país:

Al referirse el enunciado a ventas se entiende que se refiere a ventas reales. Esto es, nuestro dataframe `df_actuals`.

In [380]:
df_actuals.head()

,COUNTRY,SUBBRAND,AMOUNT,DATE
0,Portugal,Pepsi Max (L3),1.885949e+05,2023-10-01
1,Portugal,7up (L3),2.934971e+05,2023-03-01
2,Portugal,7up (L3),3.484466e+05,2023-10-01
3,Great Britain,7up Free (L3),1.172553e+06,2023-10-01
4,Norway,Pepsi Regular (L3),3.784859e+04,2023-10-01


Podemos agrupar los datos por país y ordenar para obtener un ranking de país en función de su `AMOUNT`.

In [381]:
country_distribution = (
    df_actuals
    .groupby('COUNTRY')['AMOUNT']
    .sum()
    .to_frame()
    .sort_values("AMOUNT", ascending=False)
)
country_distribution

,AMOUNT
COUNTRY,
Great Britain,3.347786e+08
Netherlands,6.395943e+07
Denmark,5.659668e+07
Norway,5.121406e+07
Italy,4.345404e+07
Hungary,4.153991e+07
Czech,3.535164e+07
Portugal,3.488807e+07
Spain,8.131266e+06


In [382]:
fig = px.bar(country_distribution, 
             x=country_distribution.index, y='AMOUNT', 
             title='Sales Volume by Country')

fig.show()

### 1.2. Distribución de ventas por mes y año:

De nuevo nuestro dataframe de actuals.

In [383]:
date_distribution = (
    df_actuals
    .groupby('DATE')['AMOUNT']
    .sum()
    .to_frame()
    .sort_values("AMOUNT", ascending=False)
)
date_distribution.head()

,AMOUNT
DATE,
2024-08-01,3.855460e+07
2023-08-01,3.817477e+07
2023-06-01,3.759999e+07
2023-05-01,3.690245e+07
2024-05-01,3.677264e+07


In [384]:
fig = px.bar(date_distribution, 
             x=date_distribution.index, y='AMOUNT', 
             title='Sales Volume by Date')

fig.show()

### 1.3. Distribución de ventas por producto:

De nuevo nuestro dataframe de actuals.

In [385]:
product_distribution = (
    df_actuals
    .groupby('SUBBRAND')['AMOUNT']
    .sum()
    .to_frame()
    .sort_values("AMOUNT", ascending=False)
)
product_distribution.head()

,AMOUNT
SUBBRAND,
Pepsi Max (L3),3.966987e+08
Pepsi Regular (L3),1.398307e+08
Lipton (L3),7.286515e+07
7up Free (L3),3.427630e+07
7up (L3),1.876851e+07


In [386]:
fig = px.bar(product_distribution, 
             x=product_distribution.index, y='AMOUNT', 
             title='Sales Volume by Subbrand')

fig.show()

### 2.1. Tendencia y estacionalidad de las ventas del país con menos ventas

El país con menos ventas es España si nos fijamos el diagrama de barras del aparatado 1.1

In [387]:
import pandas as pd

spain_mask = df_actuals.loc[:, "COUNTRY"] == "Spain"
df_actuals_spain = df_actuals.loc[spain_mask, :].reset_index(drop=True)
df_actuals_spain.head()

,COUNTRY,SUBBRAND,AMOUNT,DATE
0,Spain,7up Free (L3),83196.895876,2023-10-01
1,Spain,7up Free (L3),60318.994231,2024-01-01
2,Spain,7up Free (L3),59482.850585,2024-02-01
3,Spain,Lipton (L3),8003.513133,2024-02-01
4,Spain,7up (L3),74882.992962,2024-04-01


In [388]:
ts_spain = df_actuals_spain.groupby('DATE')['AMOUNT'].sum().reset_index()
print("Spain timeseries length:", len(ts_spain))
ts_spain.head()

Spain timeseries length: 20


,DATE,AMOUNT
0,2023-01-01,325272.938930
1,2023-02-01,295588.379605
2,2023-03-01,399710.273110
3,2023-04-01,328035.257874
4,2023-05-01,360751.156280


In [389]:
import plotly.express as px

# Create a line plot of the time series
fig = px.line(ts_spain, x='DATE', y='AMOUNT', title='Total Amount Over Time for Spain')

# Show the plot
fig.show()

Como solo tenemos mediciones de 20 meses vamos a realizar una visualización de la tendencia y la estacionalidad a 6 meses

In [390]:
from statsmodels.tsa.seasonal import seasonal_decompose

decomposition = seasonal_decompose(ts_spain['AMOUNT'], model='additive', period=6)

# Create a DataFrame to store the components
decomposed_df = pd.DataFrame({
    'DATE': ts_spain['DATE'],
    'Observed': decomposition.observed,
    'Trend': decomposition.trend,
    'Seasonal': decomposition.seasonal,
    'Residual': decomposition.resid
})
# Trend
fig_trend = px.line(decomposed_df, x='DATE', y='Trend', title='Monthly Trend Component for Spain')
fig_trend.show()

# Seasonal
fig_seasonal = px.line(decomposed_df, x='DATE', y='Seasonal', title='Monthly Seasonal Component for Spain')
fig_seasonal.show()

# Residual
fig_residual = px.line(decomposed_df, x='DATE', y='Residual', title='Monthly Residual Component for Spain')

### 2.1. Tendencia y estacionalidad de las ventas de la marca con más ventas

La marca con más ventas es Pepsi si nos fijamos el diagrama de barras del aparatado 1.2

In [391]:
import pandas as pd

pepsi_mask = (df_actuals.loc[:, "SUBBRAND"] == "Pepsi Max (L3)") | (df_actuals.loc[:, "SUBBRAND"] == "Pepsi Regular (L3)") # No me da tiempo a sumarlas
df_actuals_pepsi = df_actuals.loc[pepsi_mask, :].reset_index(drop=True)
df_actuals_pepsi.head()

,COUNTRY,SUBBRAND,AMOUNT,DATE
0,Portugal,Pepsi Max (L3),1.885949e+05,2023-10-01
1,Norway,Pepsi Regular (L3),3.784859e+04,2023-10-01
2,Norway,Pepsi Regular (L3),4.587958e+04,2023-03-01
3,Hungary,Pepsi Regular (L3),1.042737e+06,2023-03-01
4,Netherlands,Pepsi Regular (L3),4.242295e+05,2024-01-01


In [392]:
ts_pepsi = df_actuals_pepsi.groupby(['DATE'])['AMOUNT'].sum().reset_index()
print("Pepsi timeseries length:", len(ts_pepsi))
ts_pepsi.head()

Pepsi timeseries length: 20


,DATE,AMOUNT
0,2023-01-01,2.235202e+07
1,2023-02-01,2.142385e+07
2,2023-03-01,2.875809e+07
3,2023-04-01,2.487733e+07
4,2023-05-01,3.054386e+07


In [393]:
import plotly.express as px

# Create a line plot of the time series
fig = px.line(ts_pepsi, x='DATE', y='AMOUNT', title='Total Amount Over Time for Pepsi')

# Show the plot
fig.show()

Como solo tenemos mediciones de 20 meses vamos a realizar una visualización de la tendencia y la estacionalidad a 6 meses

In [394]:
from statsmodels.tsa.seasonal import seasonal_decompose

decomposition = seasonal_decompose(ts_pepsi['AMOUNT'], model='additive', period=6)

# Create a DataFrame to store the components
decomposed_df = pd.DataFrame({
    'DATE': ts_spain['DATE'],
    'Observed': decomposition.observed,
    'Trend': decomposition.trend,
    'Seasonal': decomposition.seasonal,
    'Residual': decomposition.resid
})
# Trend
fig_trend = px.line(decomposed_df, x='DATE', y='Trend', title='Monthly Trend Component for Pepsi')
fig_trend.show()

# Seasonal
fig_seasonal = px.line(decomposed_df, x='DATE', y='Seasonal', title='Monthly Seasonal Component for Pepsi')
fig_seasonal.show()

### 3. Cuales son las predicciones hechas en España y como de buenas son.

In [395]:
import pandas as pd

spain_mask = df_actuals.loc[:, "COUNTRY"] == "Spain"
df_actuals_spain = df_actuals.loc[spain_mask, :].drop(columns="COUNTRY").reset_index(drop=True)
df_actuals_spain.head()

,SUBBRAND,AMOUNT,DATE
0,7up Free (L3),83196.895876,2023-10-01
1,7up Free (L3),60318.994231,2024-01-01
2,7up Free (L3),59482.850585,2024-02-01
3,Lipton (L3),8003.513133,2024-02-01
4,7up (L3),74882.992962,2024-04-01


In [396]:
spain_mask = df_forecasts.loc[:, "COUNTRY"] == "Spain"
df_forecasts_spain = df_forecasts.loc[spain_mask, :].drop(columns="COUNTRY").reset_index(drop=True)
df_forecasts_spain.head()

,SUBBRAND,DATE,FORECAST_DATE,AMOUNT
0,7up (L3),2023-01-01,2023-12-01,52156.149274
1,7up (L3),2023-02-01,2023-01-01,65063.618870
2,7up (L3),2023-02-01,2023-12-01,52017.662876
3,7up (L3),2023-03-01,2023-01-01,63117.732848
4,7up (L3),2023-03-01,2023-02-01,68081.917712


Obtenemos los totales (también sería posible hacer un análisis parcial por marca)

In [397]:
total_actuals_spain = df_actuals_spain.groupby("DATE")["AMOUNT"].sum().reset_index()
total_actuals_spain.head()

,DATE,AMOUNT
0,2023-01-01,325272.938930
1,2023-02-01,295588.379605
2,2023-03-01,399710.273110
3,2023-04-01,328035.257874
4,2023-05-01,360751.156280


Y también obtenemos los forecasts, que en este caso son 18

In [398]:
total_forecasts_spain = df_forecasts_spain.groupby(["DATE", "FORECAST_DATE"])["AMOUNT"].sum().reset_index()
total_forecasts_spain.head()

,DATE,FORECAST_DATE,AMOUNT
0,2023-01-01,2023-12-01,322792.073635
1,2023-02-01,2023-01-01,308203.547588
2,2023-02-01,2023-12-01,325992.889561
3,2023-03-01,2023-01-01,323576.193373
4,2023-03-01,2023-02-01,301487.199998


In [401]:
import plotly.graph_objects as go

# Create a figure
fig = go.Figure()

for forecast_date in total_forecasts_spain["FORECAST_DATE"].unique():
    forecast_mask = total_forecasts_spain.loc[:, "FORECAST_DATE"] == forecast_date
    total_forecasts_spain_AIPNF = total_forecasts_spain.loc[forecast_mask, :].reset_index(drop=True)
    fig.add_trace(go.Scatter(x=total_forecasts_spain_AIPNF['DATE'], y=total_forecasts_spain_AIPNF['AMOUNT'], mode='lines', name="AI " + forecast_date.strftime("%b %Y")))

fig.add_trace(
    go.Scatter(x=total_actuals_spain['DATE'],
               y=total_actuals_spain['AMOUNT'],
               mode='lines',
               name='ACTUALS',
               line=dict(color="black")
            ))

fig.show()
